In [ ]:
drive_path = "/content/drive/MyDrive/Colab\ Notebooks/dataset.zip"
destination_path = "/content/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp $drive_path $destination_path

In [ ]:
!unzip /content/dataset.zip

Archive:  /content/dataset.zip
   creating: dataset/
   creating: dataset/annual mallow/
  inflating: dataset/annual mallow/image_02862.jpg  
  inflating: dataset/annual mallow/image_02863.jpg  
  inflating: dataset/annual mallow/image_02864.jpg  
  inflating: dataset/annual mallow/image_02865.jpg  
  inflating: dataset/annual mallow/image_02866.jpg  
  inflating: dataset/annual mallow/image_02867.jpg  
  inflating: dataset/annual mallow/image_02868.jpg  
  inflating: dataset/annual mallow/image_02869.jpg  
  inflating: dataset/annual mallow/image_02870.jpg  
  inflating: dataset/annual mallow/image_02871.jpg  
  inflating: dataset/annual mallow/image_02872.jpg  
  inflating: dataset/annual mallow/image_02873.jpg  
  inflating: dataset/annual mallow/image_02874.jpg  
  inflating: dataset/annual mallow/image_02875.jpg  
  inflating: dataset/annual mallow/image_02876.jpg  
  inflating: dataset/annual mallow/image_02877.jpg  
  inflating: dataset/annual mallow/image_02878.jpg  
  inflatin

In [ ]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
import os

In [ ]:
data_dir = "/content/dataset"
checkpoint_dir = "/content/drive/MyDrive/checkpoints/"
base_name = "resnet50_3e4_10_secondTry2_"

In [ ]:
class FlowerClassifier(nn.Module):
    def __init__(self, num_classes=25):
        super(FlowerClassifier, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)

    def forward(self, x):
        return self.backbone(x)

In [ ]:
model = FlowerClassifier(num_classes=27)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 181MB/s]


In [ ]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


full_dataset = datasets.ImageFolder(root=data_dir)

train_size = int(0.7 * len(full_dataset))
val_size = int(0.2 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform
test_dataset.dataset.transform = test_transform

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
print("Class names:")
for idx, class_name in enumerate(full_dataset.classes):
    print(f"{idx}: {class_name}")

Class names:
0: annual mallow
1: asian virginsbower
2: barbados lily
3: bull thistle
4: buttercup
5: california poppies
6: calla lily
7: canna lily
8: coltsfoot
9: common columbine
10: common cornflag
11: common daisy
12: common dandelion
13: common primroses
14: corn poppy
15: desert rose
16: fritillaries
17: garden petunia
18: passionflower
19: peruvian lily
20: scarlet beebalm
21: sunflower
22: tea roses
23: tiger lily
24: violets
25: wallflowers
26: water lillies


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [ ]:
num_epochs = 20
best_loss = float('inf')  #

In [ ]:
def test_model(model, test_loader, header):
    model.eval()  #
    correct = 0
    total = 0

    with torch.no_grad():  #
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"{header} Accuracy: {accuracy:.4f}%")
    return accuracy

In [ ]:

for epoch in range(num_epochs):
    print(f"Current Epoch: {epoch}")
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch rn: [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}")
    test_model(model, val_loader, "Validation")

    if epoch_loss < best_loss:
      accuracy_test = test_model(model, test_loader, "Test")
      best_loss = epoch_loss
      checkpoint_path = os.path.join(checkpoint_dir, f"{base_name}_epoch_{epoch+1}_accuracy_test_{accuracy_test:.4f}.pth")
      torch.save(model.state_dict(), checkpoint_path)
      print(f"Checkpoint saved at {checkpoint_path}")

print("Training complete!")

Current Epoch: 0
Epoch rn: [1/20], Train Loss: 0.9092
Validation Accuracy: 83.9483%
Test Accuracy: 82.3529%
Checkpoint saved at /content/drive/MyDrive/checkpoints/resnet50_3e4_10_secondTry2__epoch_1_accuracy_test_82.3529.pth
Current Epoch: 1
Epoch rn: [2/20], Train Loss: 0.2717
Validation Accuracy: 86.9004%
Test Accuracy: 83.4559%
Checkpoint saved at /content/drive/MyDrive/checkpoints/resnet50_3e4_10_secondTry2__epoch_2_accuracy_test_83.4559.pth
Current Epoch: 2
Epoch rn: [3/20], Train Loss: 0.2020
Validation Accuracy: 92.0664%
Test Accuracy: 93.0147%
Checkpoint saved at /content/drive/MyDrive/checkpoints/resnet50_3e4_10_secondTry2__epoch_3_accuracy_test_93.0147.pth
Current Epoch: 3
Epoch rn: [4/20], Train Loss: 0.1735
Validation Accuracy: 91.5129%
Test Accuracy: 92.2794%
Checkpoint saved at /content/drive/MyDrive/checkpoints/resnet50_3e4_10_secondTry2__epoch_4_accuracy_test_92.2794.pth
Current Epoch: 4
Epoch rn: [5/20], Train Loss: 0.1879
Validation Accuracy: 88.0074%
Current Epoch: 5

In [ ]:
!pip install onnx

In [ ]:
import torch
import os
import onnx

input_tensor = torch.rand((1, 3, 224, 224), dtype=torch.float32)


checkpoint_path = os.path.join(checkpoint_dir, 'resnet50_3e4_10_secondTry2__epoch_20_accuracy_test_97.4265.pth')

model = FlowerClassifier(num_classes=27)

model.load_state_dict(torch.load(checkpoint_path, weights_only=True))

onnx_path = checkpoint_path.replace(".pth", ".onnx")

torch.onnx.export(
    model,
    input_tensor,
    onnx_path,
    input_names=["input"],
    output_names=["output"],
    opset_version=12,
)

print(f"Model has been exported to {onnx_path}")


Model has been exported to /content/drive/MyDrive/checkpoints/resnet50_3e4_10_secondTry2__epoch_20_accuracy_test_97.4265.onnx
